In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest, pairwise, tee
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set
import functools

In [3]:
DAY = 14

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'476,71 -> 481,71\n484,52 -> 488,52\n503,41 -> 503,36 -> 503,41 -> 505,41 -> 505,34'
Split Data:
['476,71 -> 481,71',
 '484,52 -> 488,52',
 '503,41 -> 503,36 -> 503,41 -> 505,41 -> 505,34 -> 505,41 -> 507,41 -> 507,40 '
 '-> 507,41 -> 509,41 -> 509,33 -> 509,41 -> 511,41 -> 511,32 -> 511,41 -> '
 '513,41 -> 513,40 -> 513,41 -> 515,41 -> 515,31 -> 515,41 -> 517,41 -> 517,31 '
 '-> 517,41 -> 519,41 -> 519,31 -> 519,41 -> 521,41 -> 521,33 -> 521,41',
 '481,61 -> 485,61',
 '461,114 -> 461,113 -> 461,114 -> 463,114 -> 463,105 -> 463,114 -> 465,114 -> '
 '465,112 -> 465,114 -> 467,114 -> 467,105 -> 467,114 -> 469,114 -> 469,112 -> '
 '469,114 -> 471,114 -> 471,109 -> 471,114 -> 473,114 -> 473,104 -> 473,114',
 '469,101 -> 469,95 -> 469,101 -> 471,101 -> 471,96 -> 471,101 -> 473,101 -> '
 '473,96 -> 473,101 -> 475,101 -> 475,100 -> 475,101 -> 477,101 -> 477,91 -> '
 '477,101 -> 479,101 -> 479,91 -> 479,101 -> 481,101 -> 481,98 -> 481,101',
 '503,41 -> 503,36 -> 503,41 -> 505,41 -> 5

In [5]:
test_data = [
"498,4 -> 498,6 -> 496,6",
"503,4 -> 502,4 -> 502,9 -> 494,9"
]

In [6]:
def parse(str_data):
    return [[tuple(map(int, n.strip().split(","))) for n in l.split("->")] for l in str_data]

In [7]:
SAND = 1
DROP = 2
EMPTY = 0
ROCK = np.inf

In [8]:
char_dict = {EMPTY:".", ROCK:"#", SAND:"O", DROP:"+"}

In [9]:
def print_cave(cave):
    print("\n".join(["".join([char_dict[x] for x in row]) for row in cave]))

In [10]:
def make_rocks(int_data, ground=False):
    minx = np.inf
    maxx = maxy = 0
    for l in int_data:
        for a,b in l:
            minx = min(minx, a)
            maxx = max(maxx, a)
            maxy = max(maxy, b)

    cave = np.zeros((maxy + 1, (maxx + 1) - minx ))
     
    for l in int_data:
        for (x1, y1), (x2, y2) in pairwise(l):
            x1 -= minx
            x2 -= minx          
            if x1 == x2:
                y1, y2 = sorted([y1,y2])
                cave[y1:(y2+1), x1] = ROCK
            elif y1 == y2:
                x1, x2 = sorted([x1,x2])
                cave[y1, x1:(x2+1)] = ROCK
            else:
                raise ValueError
    
    cave = np.vstack((np.zeros((1, cave.shape[1])), cave))
    maxy += 1
    
    if ground:
        cave = np.hstack((cave, np.zeros((maxy + 1, maxy))))
        cave = np.hstack((np.zeros((maxy + 1, maxy)), cave))
        minx -= maxy
        maxx += maxy
    
        cave = np.vstack((cave, np.zeros((1, cave.shape[1]))))
        cave = np.vstack((cave, np.full((1, cave.shape[1]), ROCK)))
        maxy += 2
    
    return (cave, minx)

In [11]:
def place_sand(cave, py, px):
    try:
        neigh = np.array(neighbors(cave, (py, px))).T
        y_low, x_low = tuple(neigh[6:].T)
        # straight below is empty
        if cave[y_low[1], x_low[1]] == EMPTY:
            # print("empty")
            return place_sand(cave, py+1, px)
        # all non-empty, return false with new cave
        elif np.count_nonzero(cave[y_low, x_low] != EMPTY) == 3:
            # print("settle")
            cave[py,px]= SAND
            return cave, False
        # try first non-empty slow below
        else:
            i = np.argmax(cave[y_low, x_low] == 0)
            return place_sand(cave, y_low[i], x_low[i])    
    except IndexError:
        return cave, True

In [13]:
def drop_sand(cave, minx, drop_pt = (500,1)):
    dropx, dropy = drop_pt
    dropx -= minx
    cave[dropy, dropx] = 2
    
    end_found = False
    
    while not end_found:
        py, px = (dropy, dropx)
        cave, end_found = place_sand(cave, py, px)
        if cave[dropy,dropx]==SAND:
            end_found=True
    print_cave(cave)
    return np.count_nonzero(cave == SAND)

In [16]:
drop_sand(*make_rocks(parse(data)))

..................................................................................
............................................................+.....................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
....

843

In [441]:
submit_answer(DAY, 1, Out[max(Out.keys())])

True

In [17]:
drop_sand(*make_rocks(parse(data), ground=True))

......................................................................................................................................................................................................................................................................................................................................................................................................................................
......................................................................................................................................................................................................................................O...............................................................................................................................................................................................
..........................................................................................................................................................

27625

In [18]:
submit_answer(DAY, 2, Out[max(Out.keys())])

27625
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 14! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True